In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [2]:
data = pd.read_csv("./dataset.csv")



In [3]:
y = LabelEncoder().fit_transform(data["No-show"])



X = data[["Age", "Gender", "Scholarship",	"Hipertension",	"Diabetes",	"Alcoholism",	"Handcap",	"SMS_received"]]
X['Gender'] = LabelEncoder().fit_transform(X['Gender'])
X.head()

y = y.reshape((-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)
X_train = X_train.to_numpy()
X_test= X_test.to_numpy()


C:\Users\SUDARSHAN\AppData\Local\Temp\ipykernel_11000\3557131927.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gender'] = LabelEncoder().fit_transform(X['Gender'])


In [4]:
from main import Sequential, Dense
model = Sequential([
    Dense(8),
    Dense(4),
    Dense(1, activation='sigmoid')
])

In [5]:
model.compile(loss='binary_cross_entropy')

In [6]:
X_train.shape

(77368, 8)

In [7]:
model.fit(X_train, y_train, epochs=100, learning_rate=1e-2)

Epoch: 1, time taken: 1.7309131622314453
Epoch: 2, time taken: 1.7918312549591064
Epoch: 3, time taken: 1.8032786846160889
Epoch: 4, time taken: 1.8565220832824707
Epoch: 5, time taken: 1.8534443378448486
Epoch: 6, time taken: 1.8417119979858398
Epoch: 7, time taken: 1.656327724456787
Epoch: 8, time taken: 1.8472867012023926
Epoch: 9, time taken: 1.8622667789459229
Epoch: 10, time taken: 1.884401559829712
Epoch: 11, time taken: 1.7068023681640625
Epoch: 12, time taken: 1.830345869064331
Epoch: 13, time taken: 2.0371859073638916
Epoch: 14, time taken: 2.1447322368621826
Epoch: 15, time taken: 2.674138307571411
Epoch: 16, time taken: 2.1815176010131836
Epoch: 17, time taken: 1.9535448551177979
Epoch: 18, time taken: 1.8759486675262451
Epoch: 19, time taken: 1.7535533905029297
Epoch: 20, time taken: 1.889014482498169
Epoch: 21, time taken: 1.9420957565307617
Epoch: 22, time taken: 1.968411922454834
Epoch: 23, time taken: 1.9044325351715088
Epoch: 24, time taken: 1.8613665103912354
Epoch: 

In [31]:
# model.fit(X_train, y_train, epochs=500, learning_rate=1e-2)


In [8]:
y_pred = model.predict(X_test)

In [9]:
from main import accuracy
accuracy(y_pred>0.5, y_test)

np.float64(0.7970988268645014)

In [10]:
for i in y_pred[0]:
    print(i)

0.3431997077832827
0.3530917814661179
0.34957905653405286
0.35508572018298906
0.35091410585898064
0.3507057932684431
0.35871470318098997
0.341629267324719
0.3510176835964448
0.33870365356528187
0.35949006193581917
0.35915292566187784
0.3493720833175844
0.36005943400101337
0.36012838224677196
0.3575433363079586
0.3574851005066812
0.3538887479748249
0.3261251443193898
0.33530847313449613
0.36104369992691326
0.35428753616953684
0.3491192628317189
0.3400622923419816
0.34541351385544994
0.3467573694310024
0.3546865270563961
0.36205503003515777
0.3530917814661179
0.3611279268824686
0.3359129491338128
0.35110294376846635
0.35241769885306273
0.36032962085472947
0.3594986410085223
0.3587517036532111
0.3566845060008737
0.34581800921601136
0.34647297507007785
0.3514433124648615
0.3374351159726791
0.3514433124648615
0.3598916462616321
0.3396710931919087
0.34728483765039037
0.3390943106830289
0.3453743636148312
0.34695294740817445
0.3548959117325698
0.3294428675975671
0.3554074279898273
0.350181383